<a href="https://colab.research.google.com/github/johkaizer/johkaizer.github.io/blob/master/FS%2BClustering_MESTRADO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#carregar dados

import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = pd.read_csv("/content/vendizap_esparsa.csv", sep=';', encoding='ISO-8859-1', usecols=list(range(5, 104)))
#df = pd.read_csv("/content/online_retail_II_matrix.csv", sep=';', encoding='ISO-8859-1', nrows=10000)
df2 = pd.read_csv("/content/vendizap_esparsa.csv", sep=';', encoding='ISO-8859-1', usecols=list(range(2, 3)))
# função para substituir vírgulas por pontos e converter para float
def str_to_float(x):
    if isinstance(x, str):
        return float(x.replace(',', '.'))
    else:
        return x

def print_best_worst (scores, vezes=5):
    scores = sorted(scores, reverse = True)
    
    print(f"The {vezes} best features selected by this method are :")
    for i in range(vezes):
        print(scores[i][1])
    
    print (f"The {vezes} worst features selected by this method are :")
    for i in range(vezes):
        print(scores[len(scores)-1-i][1])
df = df.applymap(str_to_float).astype(float)
df['Soma'] = df.sum(axis=1)
df=df.fillna(0)
X = df.drop(columns=["Soma"]).values
y = df2["Bairro"].values
scaler = StandardScaler()
X = scaler.fit_transform(X)



In [ ]:
# Calcula a matriz de correlação
corr_matrix = df.corr()

# Verifica as correlações com a variável de destino
correlations = corr_matrix['Soma'].drop('Soma')

# Imprime as correlações
#print(correlations[correlations > 0.1].sort_values(ascending=False))

In [3]:
!pip install sklearn-genetic
def elbow(importances, atributos):
  # Ordenar as importâncias de forma decrescente
  sorted_importances = sorted(zip(importances, atributos), reverse=True)
  # Calcular a variância explicada acumulada para cada número de atributos mantidos
  variances = []
  total_variance = np.sum(importances)
  cumulative_variance = 0
  for i in range(len(atributos)):
      var = sorted_importances[i][0]
      cumulative_variance += var
      variances.append(cumulative_variance / total_variance)

  # Plotar a curva de variância em função do número de atributos mantidos
  fig, ax = plt.subplots()
  ax.plot(variances, 'o-')
  ax.set_xlabel('Número de atributos')
  ax.set_ylabel('Variância explicada acumulada')
  plt.show()

  # Selecionar o número de atributos que explique uma quantidade significativa da variância
  threshold = 0.8
  num_features = np.sum(np.array(variances) < threshold) + 1
  selected_features = [f[1] for f in sorted_importances[:num_features]]

  print(f'Foram selecionados {num_features} atributos: {selected_features}')
  return num_features


def getScores(importance, atributos):
  scores = []
  for i in range(len(df.drop(columns=["Soma"]).columns)):
      scores.append((importance[i],atributos[i]))
  #print_best_worst(scores)
  return scores


def unsupervised(X, y, algoritmo='randomforestregressor'): 
  if algoritmo=='randomforestregressor':
    from sklearn.ensemble import RandomForestRegressor
    rf = RandomForestRegressor()
  elif algoritmo=='randomforestclassifier':
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier()
  elif algoritmo=='lgbmregressor':
    import lightgbm as lgb 
    rf = lgb.LGBMRegressor()
  elif algoritmo=='lgbmclassifier':
    import lightgbm as lgb 
    rf = lgb.LGBMClassifier()
  elif algoritmo=='xgboostregressor':
    from xgboost import XGBRegressor
    rf = XGBRegressor(objective='reg:squarederror')
  
    
  rf.fit(X, y)
  # Obtendo as importâncias dos atributos
  importances = rf.feature_importances_
  atributos=df.drop(columns=["Soma"]).columns
  #Elbow para saber o numero de atributos a serem usados
  elbow(importances, atributos)
  return getScores(importances, atributos) #retorna os scores

def others(X, y, algoritmo='svm'): #X=df.drop(columns=["Soma"]).values
  if algoritmo=='svm':
    from sklearn import svm
    rf = svm.SVC(kernel='linear')
    rf.fit(X, y)
    importances = pd.Series(abs(rf.coef_[0]), index=df.drop(columns=["Soma"]).columns)
  elif algoritmo=='selectkbest':
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2, mutual_info_classif
    rf = SelectKBest(score_func=chi2, k='all')
    rf.fit(X, y)
    importances = pd.Series(rf.scores_, index=df.drop(columns=["Soma"]).columns)
  elif algoritmo=='rfe':
    from sklearn.feature_selection import RFE
    from sklearn.linear_model import LogisticRegression
    rf = RFE(LogisticRegression())
    rf.fit(X, y)
    importances = pd.Series(rf.ranking_, index=df.drop(columns=["Soma"]).columns)
  elif algoritmo=='genetico': ##Usar se tiver classes
    from genetic_selection import GeneticSelectionCV
    from sklearn.linear_model import LogisticRegression
    from sklearn.model_selection import StratifiedKFold, train_test_split
    from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.2,
                                                    random_state=42)
    scv = StratifiedKFold(n_splits=2)
    # Definindo o classificador
    clf = RandomForestClassifier()
    selector = GeneticSelectionCV(clf,
                                  cv=scv,
                                  verbose=0,
                                  scoring="f1_macro",
                                  max_features=15,
                                  n_population=50,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=40,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True)
    selector = selector.fit(X_train, y_train)
    # get the selected features
    cols = df.drop(columns=["Soma"]).columns
    selected_feats = [cols[i] for i in np.where(selector.support_)[0]]
    print(selected_feats)
    importances=selector.support_
    print("<DESCONSIDERAR ELBOW E SCORES!!!>")

  elif algoritmo=='lasso':
    from sklearn.feature_selection import SelectFromModel
    from sklearn.linear_model import Lasso
    rf = Lasso(alpha=0.15)
    sfm = SelectFromModel(rf)
    X_selected = sfm.fit_transform(X, y)
    rf.fit(X_selected, y)
    coeficientes = rf.coef_
    # obtendo os nomes dos recursos selecionados
    selected_features = np.where(sfm.get_support())[0]
    feature_names = df.drop(columns=["Soma"]).columns[selected_features]
    selecionados=sorted(zip(feature_names, coeficientes), key=lambda x: abs(x[1]), reverse=True)
    importances = [0] * len(df.drop(columns=["Soma"]).columns)
    for i, idx in enumerate(selected_features):
        importances[idx] = coeficientes[i]
  
  atributos=df.drop(columns=["Soma"]).columns
  #Elbow para saber o numero de atributos a serem usados
  elbow(importances, atributos)
  return getScores(importances, atributos) #retorna os scores

def criterio_kaiser(X):
  ###Determinar numero de componentes: método do critério de Kaiser
  # Calcula a matriz de covariância
  cov_mat = np.cov(X.T)
  # Calcula os autovalores e autovetores da matriz de covariância
  eig_vals, eig_vecs = np.linalg.eig(cov_mat)
  # Ordena os autovalores em ordem decrescente
  sorted_eig_vals = sorted(eig_vals, reverse=True)
  # Aplica o critério de Kaiser
  num_components = len([eig_val for eig_val in sorted_eig_vals if eig_val > 1])
  print("Número de componentes pelo criterio de kaiser: ", num_components)
  return num_components

def pca(X, y, criterio='kaizer'):
  from sklearn.decomposition import PCA
  from sklearn.metrics import mean_squared_error

  if criterio=='kaizer':
    #Definindo o numero de componentes pelo criterio de Kaiser
    num_components=criterio_kaiser(X)
  elif criterio=='var':
    ###Determinar numero de componentes: método do limiar
    pca = PCA().fit(X)
    explained_variance = pca.explained_variance_ratio_
    cumulative_explained_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
    #Plotar
    plt.plot(range(1, pca.n_components_ + 1), cumulative_explained_variance_ratio, marker='o')
    plt.xlabel('Número de componentes principais')
    plt.ylabel('Variância explicada cumulativa')
    plt.show()
    #Aplicar limiar
    limite_variancia = 0.8
    num_components=len([x for x in cumulative_explained_variance_ratio if x < limite_variancia])
    print(f"Número de componentes selecionado: {num_components}")

  # aplicando a técnica PCA para selecionar os atributos
  pca = PCA(n_components=num_components)
  X_selected = pca.fit_transform(X)
  print(f"Variancia explicada: {sum(pca.explained_variance_ratio_)}")
  return X_selected

def ica(X, y):
  from sklearn.decomposition import FastICA
  import matplotlib.pyplot as plt
  from scipy.stats import kurtosis

  limiar=0.8
  ica = FastICA(n_components=X.shape[1])
  S_ = ica.fit_transform(X)
  # calculando a variância explicada por cada componente
  explained_variance = np.var(S_, axis=0)
  # calculando a variância total
  total_variance = np.sum(explained_variance)
  # calculando a proporção de variância explicada acumulada por cada componente
  explained_variance_ratio = explained_variance / total_variance
  # calculando a variância explicada acumulada
  cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
  # plotando a curva de variância explicada acumulada
  import matplotlib.pyplot as plt
  plt.plot(range(1, X.shape[1]+1), cumulative_variance_ratio)
  plt.xlabel('Número de componentes')
  plt.ylabel('Variância explicada acumulada')
  plt.show()
  # selecionando o número de componentes que explicam, pelo menos, limiar% da variância
  num_components = np.argmax(cumulative_variance_ratio >= limiar) + 1
  print(f"Número de componentes selecionado: {num_components}")

  # aplicando a técnica PCA para selecionar os atributos
  ica = FastICA(n_components=num_components)
  X_selected = ica.fit_transform(X)
  return X_selected

def nmf(X, y): #X=df.drop(columns=["Soma"]).values
  from sklearn.decomposition import NMF
  import matplotlib.pyplot as plt
  from scipy.stats import kurtosis

  limiar=0.8
  nmf = NMF(n_components=X.shape[1])
  S_ = nmf.fit_transform(X)
  # calculando a variância explicada por cada componente
  explained_variance = np.var(S_, axis=0)
  # calculando a variância total
  total_variance = np.sum(explained_variance)
  # calculando a proporção de variância explicada acumulada por cada componente
  explained_variance_ratio = explained_variance / total_variance
  # calculando a variância explicada acumulada
  cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
  # plotando a curva de variância explicada acumulada
  import matplotlib.pyplot as plt
  plt.plot(range(1, X.shape[1]+1), cumulative_variance_ratio)
  plt.xlabel('Número de componentes')
  plt.ylabel('Variância explicada acumulada')
  plt.show()
  # selecionando o número de componentes que explicam, pelo menos, limiar% da variância
  num_components = np.argmax(cumulative_variance_ratio >= limiar) + 1
  print(f"Número de componentes selecionado: {num_components}")
  
  # aplicando a técnica nmf para selecionar os atributos
  nmf = NMF(n_components=num_components)
  X_selected = nmf.fit_transform(X)
  return X_selected



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.5 MB/s eta 0:00:00


In [4]:
def kmeans(X):
  from sklearn.cluster import KMeans
  from sklearn.metrics import silhouette_score, calinski_harabasz_score
  import numpy as np

  # Define o intervalo de número de clusters para testar
  n_clusters_range = range(2, X.shape[1])

  # Inicializa as listas para armazenar os resultados
  silhouette_scores = []
  calinski_harabasz_scores = []
  sse = []

  # Itera sobre o número de clusters para testar
  for n_clusters in n_clusters_range:
      # Executa o algoritmo Sparse K-Means
      kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=100, n_init=1)
      kmeans.fit(X)
      labels = kmeans.labels_
      
      # Calcula os scores de silhouette e calinski-harabasz
      s_score = silhouette_score(X, labels, metric='cosine')
      ch_score = calinski_harabasz_score(X, labels)
      
      # Adiciona os scores nas listas
      silhouette_scores.append(s_score)
      calinski_harabasz_scores.append(ch_score)

      # Calcula o erro quadrático médio e a variância explicada acumulada
      sse.append(kmeans.inertia_)


  # Encontra o número de clusters ideal
  s_best_idx = np.argmax(silhouette_scores)
  ch_best_idx = np.argmax(calinski_harabasz_scores)
  sse_best_idx = np.argmin(sse)

  # Imprime o número de clusters ideais e os scores correspondentes
  print(f'Número ideal de clusters (silhouette score): {n_clusters_range[s_best_idx]}, Score: {silhouette_scores[s_best_idx]}')
  print(f'Número ideal de clusters (calinski-harabasz score): {n_clusters_range[ch_best_idx]}, Score: {calinski_harabasz_scores[ch_best_idx]}')
  print(f'Número ideal de clusters (SSE): {n_clusters_range[sse_best_idx]}, Score: {sse[sse_best_idx]}')

  kmeans = KMeans(n_clusters=n_clusters_range[s_best_idx], init='k-means++')
  kmeans.fit(X)
  labels = kmeans.labels_

  ext=pd.DataFrame(labels)
  ext.to_excel("outputK.xlsx")

def dbscan(X):
    from sklearn.cluster import DBSCAN
    from sklearn.metrics import silhouette_score, calinski_harabasz_score
    import numpy as np

    # Define o intervalo de parâmetros para testar
    eps_range = np.linspace(0.1, 1.0, 10)
    min_samples_range = range(2, X.shape[1])

    # Inicializa as listas para armazenar os resultados
    silhouette_scores = []
    calinski_harabasz_scores = []

    # Itera sobre os valores de eps e min_samples para testar
    for eps in eps_range:
        for min_samples in min_samples_range:
            # Executa o algoritmo DBSCAN
            dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine')
            dbscan.fit(X)
            labels = dbscan.labels_

            # Verifica se o número de clusters é maior que 1 para calcular os scores
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            if n_clusters > 1:
                # Calcula os scores de silhouette e calinski-harabasz
                s_score = silhouette_score(X, labels, metric='cosine')
                ch_score = calinski_harabasz_score(X, labels)

                # Adiciona os scores nas listas
                silhouette_scores.append(s_score)
                calinski_harabasz_scores.append(ch_score)
    # Encontra os parâmetros ideais
    s_best_idx = np.argmax(silhouette_scores)
    ch_best_idx = np.argmax(calinski_harabasz_scores)

    # Imprime os parâmetros ideais e os scores correspondentes
    print(f'Parâmetros ideais (silhouette score): eps={eps_range[s_best_idx//len(min_samples_range)],} min_samples={min_samples_range[s_best_idx%len(min_samples_range)]}, Score: {silhouette_scores[s_best_idx]}')
    print(f'Parâmetros ideais (calinski-harabasz score): eps={eps_range[ch_best_idx//len(min_samples_range)],} min_samples={min_samples_range[ch_best_idx%len(min_samples_range)]}, Score: {calinski_harabasz_scores[ch_best_idx]}')

    dbscan = DBSCAN(eps=eps_range[s_best_idx//len(min_samples_range)], min_samples=min_samples_range[s_best_idx%len(min_samples_range)], metric='cosine')
    dbscan.fit(X)
    labels= dbscan.labels_
    # Imprime o número de clusters e os scores correspondentes
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    print(f'Número de clusters: {n_clusters}') 

    ext=pd.DataFrame(labels)
    ext.to_excel("outputD.xlsx")

    return labels


In [5]:
def hierarchical_clustering(X):
    from sklearn.cluster import AgglomerativeClustering
    from sklearn.metrics import silhouette_score, calinski_harabasz_score
    # Define o intervalo de número de clusters para testar
    n_clusters_range = range(2, X.shape[1])

    # Inicializa as listas para armazenar os resultados
    silhouette_scores = []
    calinski_harabasz_scores = []

    # Itera sobre o número de clusters para testar
    for n_clusters in n_clusters_range:
        # Executa o algoritmo Hierarchical Clustering
        hc = AgglomerativeClustering(n_clusters=n_clusters)
        hc.fit(X)
        labels = hc.labels_

        # Calcula os scores de silhouette e calinski-harabasz
        s_score = silhouette_score(X, labels, metric='cosine')
        ch_score = calinski_harabasz_score(X, labels)

        # Adiciona os scores nas listas
        silhouette_scores.append(s_score)
        calinski_harabasz_scores.append(ch_score)

    # Encontra o número de clusters ideal
    s_best_idx = np.argmax(silhouette_scores)
    ch_best_idx = np.argmax(calinski_harabasz_scores)

    # Imprime o número de clusters ideais e os scores correspondentes
    print(f'Número ideal de clusters (silhouette score): {n_clusters_range[s_best_idx]}, Score: {silhouette_scores[s_best_idx]}')
    print(f'Número ideal de clusters (calinski-harabasz score): {n_clusters_range[ch_best_idx]}, Score: {calinski_harabasz_scores[ch_best_idx]}')

    hc = AgglomerativeClustering(n_clusters=n_clusters_range[s_best_idx])
    hc.fit(X)
    labels = hc.labels_

    ext=pd.DataFrame(labels)
    ext.to_excel("outputH.xlsx")


In [6]:
def espectral(X):
  from sklearn.cluster import SpectralClustering
  from sklearn.metrics import silhouette_score, calinski_harabasz_score
  from sklearn.preprocessing import normalize

  # Define o intervalo de número de clusters para testar
  n_clusters_range = range(2, X.shape[1])

  # Inicializa as listas para armazenar os resultados
  silhouette_scores = []
  calinski_harabasz_scores = []

  # Itera sobre o número de clusters para testar
  for n_clusters in n_clusters_range:
      # Executa o algoritmo de Clustering Espectral
      spectral = SpectralClustering(n_clusters=n_clusters, affinity='nearest_neighbors')
      spectral.fit(X)
      labels = spectral.labels_
      
      # Calcula o score de silhouette
      s_score = silhouette_score(X, labels)
      ch_score = calinski_harabasz_score(X, labels)
      
      # Adiciona o score na lista
      silhouette_scores.append(s_score)
      calinski_harabasz_scores.append(ch_score)

  # Encontra o número de clusters ideal
  s_best_idx = np.argmax(silhouette_scores)
  ch_best_idx = np.argmax(calinski_harabasz_scores)

  # Imprime o número de clusters ideais e os scores correspondentes
  print(f'Número ideal de clusters (silhouette score): {n_clusters_range[s_best_idx]}, Score: {silhouette_scores[s_best_idx]}')
  print(f'Número ideal de clusters (calinski-harabasz score): {n_clusters_range[ch_best_idx]}, Score: {calinski_harabasz_scores[ch_best_idx]}')
  
  spectral = SpectralClustering(n_clusters=n_clusters_range[best_idx], affinity='nearest_neighbors')
  spectral.fit(X)
  labels = spectral.labels_

  ext=pd.DataFrame(labels)
  ext.to_excel("outputE.xlsx")



In [7]:
def fuzzy(X):
  !pip install fuzzy-c-means
  from sklearn.metrics import silhouette_score, calinski_harabasz_score
  from sklearn.preprocessing import normalize
  from sklearn.utils.extmath import randomized_svd
  import numpy as np
  from fcmeans import FCM

  # Define o intervalo de número de clusters para testar
  n_clusters_range = range(2, X.shape[1])

  # Inicializa as listas para armazenar os resultados
  silhouette_scores = []
  calinski_harabasz_scores = []

  # Itera sobre o número de clusters para testar
  for n_clusters in n_clusters_range:
      # Executa o algoritmo Fuzzy C-Means
      fcm = FCM(n_clusters=n_clusters)
      fcm.fit(X)
      labels = fcm.u.argmax(axis=1)
      
      # Calcula os scores de silhouette e calinski-harabasz
      s_score = silhouette_score(X, labels)
      ch_score = calinski_harabasz_score(X, labels)
      
      # Adiciona os scores nas listas
      silhouette_scores.append(s_score)
      calinski_harabasz_scores.append(ch_score)

  # Encontra o número de clusters ideal
  s_best_idx = np.argmax(silhouette_scores)
  ch_best_idx = np.argmax(calinski_harabasz_scores)

  # Imprime o número de clusters ideais e os scores correspondentes
  print(f'Número ideal de clusters (silhouette score): {n_clusters_range[s_best_idx]}, Score: {silhouette_scores[s_best_idx]}')
  print(f'Número ideal de clusters (calinski-harabasz score): {n_clusters_range[ch_best_idx]}, Score: {calinski_harabasz_scores[ch_best_idx]}')

  fcm = FCM(n_clusters=n_clusters_range[s_best_idx])
  fcm.fit(X)
  labels = fcm.u.argmax(axis=1)

  fcm_labels = fcm.predict(X)
  ext=pd.DataFrame(fcm_labels)
  ext.to_excel("outputF.xlsx")
